In [2]:
import pandas as pd
import os
from typing import List
import json
import numpy as np

In [3]:
anos = [i for i in range(2014, 2023)]

In [4]:
def get_files_to_read(ano: int) -> List[str]:
    """
    Retorna uma lista com os nomes dos arquivos que devem ser lidos de acordo com o ano fornecido.

    Args:
        ano (int): O ano para o qual se deseja obter os arquivos.

    Returns:
        List[str]: Uma lista de strings contendo os nomes dos arquivos a serem lidos.
    """
    if ano in (2014, 2015, 2016, 2017):
        return [f'{ano}_scouts_raw.csv']
    elif ano in (2018, 2019, 2020, 2022):
        return [f'rodada-{i}.csv' for i in range(0, 39)]
    elif ano in (0, 2021):
        return [f'Mercado_{i}.txt' for i in range(1, 40)]


In [5]:
files = {}

for ano in anos:
    # Cria uma lista vazia no dicionário `files` para o ano atual
    files[ano] = get_files_to_read(ano)

In [6]:
datasets = dict()

In [7]:
def read_json_file(file_path: str) -> pd.DataFrame:
    """
    Lê um arquivo JSON e retorna um DataFrame do pandas com os dados do arquivo.

    Args:
        file_path (str): O caminho completo para o arquivo JSON a ser lido.

    Returns:
        pd.DataFrame: Um DataFrame do pandas com os dados do arquivo.
    """
    with open(file_path, 'r', encoding='ISO-8859-1') as f:
        json_data = json.loads(f.read())
    for at in json_data['atletas']:
        at.update(at.pop('scout'))

    atletas = pd.DataFrame(json_data['atletas'], columns=json_data['atletas'][0].keys())
    times = pd.DataFrame(json_data['clubes'], columns=json_data['clubes']['1371'].keys())

    times.rename(columns={'nome': 'clube_nome'}, inplace=True)

    atletas = atletas.merge(
        times[['id', 'clube_nome']],
        left_on=['clube_id'],
        right_on=['id'],
        how='left'
    )
    

    return atletas

def concat_in_dataframe(list_of_files: List[str], func, ano: int, path: str) -> pd.DataFrame:
    """
    Lê vários arquivos e retorna um DataFrame do pandas concatenando os DataFrames de cada arquivo.

    Args:
        list_of_files (List[str]): Uma lista de strings contendo os nomes dos arquivos a serem lidos.
        func (function): Uma função que será aplicada a cada arquivo para convertê-lo em um DataFrame.
        ano (int): O ano a ser usado para ler os arquivos.

    Returns:
        pd.DataFrame: Um DataFrame do pandas que contém os dados de todos os arquivos.
    """
    if ano in (2014, 2015, 2016):
        jogadores = pd.read_csv(f'{path}/{ano}/{ano}_jogadores.csv')
        jogadores['status_id'] = 0
        times = pd.read_csv(f'{path}/{ano}/{ano}_times.csv')

    data = pd.DataFrame()
    for file in list_of_files:
        try:
            file_data = func(f'{path}/{ano}/{file}')
            if ano in (2014, 2015, 2016):
                file_data = file_data.merge(
                    jogadores[['ID', 'Apelido', 'PosicaoID']],
                    left_on=['AtletaID'],
                    right_on=['ID'],
                    how='left'
                )
                file_data = file_data.merge(
                    times[['ID', 'Nome']],
                    left_on=['ClubeID'],
                    right_on=['ID'],
                    how='left'
                )

            data = pd.concat([data, file_data], ignore_index=True)
        except FileNotFoundError:
            print(f'Arquivo {file} não existe no ano {ano}')
        except UnicodeDecodeError:
            print(f'Erro de codificação no arquivo {file} do ano {ano}')
    return data

In [8]:
path = 'data/01_raw'

for ano in anos:
    func = pd.read_csv if ano != 2021 else read_json_file
    datasets[ano] = concat_in_dataframe(files[ano], func, ano, path)

Arquivo rodada-0.csv não existe no ano 2018
Arquivo rodada-0.csv não existe no ano 2019
Arquivo rodada-0.csv não existe no ano 2020


In [9]:
scouts = list()
for ano, dataset in datasets.items():
    colunas = list(dataset.columns)
    for name in colunas:
        if len(name) <= 2 and name not in scouts:
            scouts.append(name)
    print({ano: [i for i in colunas if i not in scouts]})

{2014: ['AtletaID', 'Rodada', 'ClubeID', 'Participou', 'Posicao', 'Jogos', 'Pontos', 'PontosMedia', 'Preco', 'PrecoVariacao', 'Partida', 'Mando', 'Titular', 'Substituido', 'TempoJogado', 'Nota', 'ID_x', 'Apelido', 'PosicaoID', 'ID_y', 'Nome']}
{2015: ['Rodada', 'ClubeID', 'AtletaID', 'Jogos', 'Pontos', 'PontosMedia', 'Preco', 'PrecoVariacao', 'ID_x', 'Apelido', 'PosicaoID', 'ID_y', 'Nome']}
{2016: ['Rodada', 'ClubeID', 'AtletaID', 'Participou', 'Pontos', 'PontosMedia', 'Preco', 'PrecoVariacao', 'ID_x', 'Apelido', 'PosicaoID', 'ID_y', 'Nome']}
{2017: ['Unnamed: 0', 'athletes.atletas.scout', 'atletas.apelido', 'atletas.atleta_id', 'atletas.clube.id.full.name', 'atletas.clube_id', 'atletas.foto', 'atletas.jogos_num', 'atletas.media_num', 'atletas.nome', 'atletas.pontos_num', 'atletas.posicao_id', 'atletas.preco_num', 'Rodada', 'atletas.status_id', 'atletas.variacao_num']}
{2018: ['Unnamed: 0', 'atletas.nome', 'atletas.slug', 'atletas.apelido', 'atletas.foto', 'atletas.atleta_id', 'atletas

In [12]:
def unificar_colunas(df):
    # Mapeamento de colunas semelhantes
    mapeamento_colunas = {
        'AtletaID': 'atleta_id',
        'athletes.atleta_id': 'atleta_id',
        'atletas.atleta_id': 'atleta_id',
        'atletas.apelido': 'apelido',
        # 'atletas.slug': 'apelido',
        'Apelido': 'apelido',
        'Rodada': 'rodada_id',
        'atletas.rodada_id': 'rodada_id',
        'ClubeID': 'clube_id',
        'atletas.clube_id': 'clube_id',
        'Nome': 'clube_nome',
        'atletas.clube.id.full.name': 'clube_nome',
        # 'Posicao': 'posicao_id',
        'atletas.posicao_id': 'posicao_id',
        'PosicaoID': 'posicao_id',
        # 'Jogos': 'jogos_num',
        # 'atletas.jogos_num': 'jogos_num',
        'Pontos': 'pontos_num',
        'atletas.pontos_num': 'pontos_num',
        'PontosMedia': 'media_num',
        'atletas.media_num': 'media_num',
        'Preco': 'preco_num',
        'atletas.preco_num': 'preco_num',
        'PrecoVariacao': 'variacao_num',
        'atletas.variacao_num': 'variacao_num',
        # 'atletas.foto': 'foto',
        'status_id': 'status_id',
        'atletas.status_id': 'status_id',
        # Incluir outras colunas em comum aqui
    }
    
    # Renomear as colunas usando o mapeamento
    df = df.rename(columns={coluna_antiga: mapeamento_colunas[coluna_antiga] for coluna_antiga in df.columns if coluna_antiga in mapeamento_colunas})

    if 'status_id' not in df.columns:
        df['status_id'] = None


    resto = [i for i in df.columns if i not in mapeamento_colunas.values() and i not in scouts]

    # Manter apenas as colunas em comum
    for scout in scouts:
        if scout not in df.columns:
            df[scout] = None
    df = df[list(set(mapeamento_colunas.values())) + scouts]
    return df, resto

In [13]:
concatenado = pd.DataFrame()

for ano in anos:
    new_df, resto = unificar_colunas(datasets[ano])
    new_df['ano'] = ano
    print(f"Ano: {ano}")
    print(f"Colunas diferentes: {resto}\n")
    concatenado = pd.concat([concatenado, new_df], ignore_index=True)

Ano: 2014
Colunas diferentes: ['Participou', 'Posicao', 'Jogos', 'Partida', 'Mando', 'Titular', 'Substituido', 'TempoJogado', 'Nota', 'ID_x', 'ID_y']

Ano: 2015
Colunas diferentes: ['Jogos', 'ID_x', 'ID_y']

Ano: 2016
Colunas diferentes: ['Participou', 'ID_x', 'ID_y']

Ano: 2017
Colunas diferentes: ['Unnamed: 0', 'athletes.atletas.scout', 'atletas.foto', 'atletas.jogos_num', 'atletas.nome']

Ano: 2018
Colunas diferentes: ['Unnamed: 0', 'atletas.nome', 'atletas.slug', 'atletas.foto']

Ano: 2019
Colunas diferentes: ['Unnamed: 0', 'atletas.nome', 'atletas.slug', 'atletas.foto']

Ano: 2020
Colunas diferentes: ['Unnamed: 0', 'atletas.nome', 'atletas.slug', 'atletas.foto', 'atletas.jogos_num', 'athletes$atletas$scout']

Ano: 2021
Colunas diferentes: ['jogos_num', 'slug', 'apelido_abreviado', 'nome', 'foto']

Ano: 2022
Colunas diferentes: ['Unnamed: 0', 'atletas.jogos_num', 'atletas.minimo_para_valorizar', 'atletas.slug', 'atletas.apelido_abreviado', 'atletas.nome', 'atletas.foto', 'atletas.g

In [14]:
concatenado = concatenado.replace({None: np.nan})

# concatenado = concatenado.dropna(how='all', subset=(['FS', 'PE', 'A', 'FT', 'FD', 'FF', 'G', 'I', 'PP', 'RB', 'FC', 'GC', 'CA', 'CV', 'SG', 'DD', 'DP', 'GS', 'PI', 'DS', 'DE', 'PC', 'PS']))

In [15]:
def preencher_nulos(df):
    # Lista de colunas que contêm as estatísticas dos atletas
    colunas_estatisticas = ['FS', 'PE', 'A', 'FT', 'FD', 'FF', 'G', 'I', 'PP', 'RB', 'FC', 'GC', 'CA', 'CV', 'SG', 'DD', 'DP', 'GS', 'PI', 'DS', 'DE', 'PC', 'PS']

    # Ordena o DataFrame por ano, atleta_id e rodada_id
    df = df.sort_values(['ano', 'atleta_id', 'rodada_id'])

    # Preenche os valores nulos com o valor da rodada anterior para cada atleta e ano
    df[colunas_estatisticas] = df.groupby(['ano', 'atleta_id'])[colunas_estatisticas].fillna(method='ffill')

    return df

In [16]:
def desacumular_rodadas(df):
    # Lista das colunas que precisam ser desacumuladas
    colunas_desacumular = ['FS', 'PE', 'A', 'FT', 'FD', 'FF', 'G', 'I', 'PP', 'RB', 'FC', 'GC', 'CA', 'CV', 'SG', 'DD', 'DP', 'GS', 'PI', 'DS', 'DE', 'PC', 'PS']
    
    # Ordene o DataFrame por ano, atleta_id e rodada_id
    df = df.sort_values(by=['ano', 'atleta_id', 'rodada_id']).reset_index(drop=True)
    
    # Crie um novo DataFrame para armazenar os resultados
    df_desacumulado = df.copy()
    
    # Desacumule as colunas
    for coluna in colunas_desacumular:
        df_desacumulado[coluna] = df.groupby(['ano', 'atleta_id'])[coluna].diff().fillna(df[coluna])

    return df_desacumulado


In [17]:
concatenado[
        (concatenado['atleta_id'] == 114648) &
        (concatenado['ano'] == 2021)
    ][['rodada_id', 'FS', 'PE', 'A', 'FT', 'FD', 'FF', 'G', 'I', 'PP', 'FC', 'GC', 'CA', 'CV', 'SG', 'DD', 'DP', 'GS', 'PI', 'DS']]

,rodada_id,FS,PE,A,FT,FD,FF,G,I,PP,FC,GC,CA,CV,SG,DD,DP,GS,PI,DS
233210,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
233378,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234804,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235602,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236448,25,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,1.0
236774,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238076,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238811,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239708,29,7.0,NaN,NaN,NaN,1.0,2.0,2.0,NaN,NaN,6.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,39.0,1.0
240472,30,8.0,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,8.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,46.0,1.0


In [18]:
# Descobrir quais anos precisam desacumular

for ano in anos:
    print(ano, concatenado[concatenado['ano'] == ano]['G'].max())

2014 3.0
2015 19.0
2016 3.0
2017 18.0
2018 18.0
2019 25.0
2020 113.0
2021 18.0
2022 25.0


In [19]:
def desacumular_anos(df, anos):
    desacumular = df[
            (df['ano'].isin(anos))
        ]

    preenchido = preencher_nulos(desacumular)
    desacumulado = desacumular_rodadas(preenchido)
    
    return pd.concat(
        [
            df[
                ~(df['ano'].isin(anos))
            ],
            desacumulado
        ]
    )

In [20]:
final_data = desacumular_anos(
    concatenado,
    anos = [2015, 2017, 2018, 2019, 2020, 2021, 2022]
    )

In [21]:
pontuacao_por_ano = {
    2014: {
        'RB': 1.7, 'FC': -0.5, 'GC': -6.0, 'CA': -2.0, 'CV': -5.0, 'FS': 0.5, 'PE': -0.3, 'FT': 3.5,
        'FD': 1.0, 'FF': 0.7, 'G': 8.0, 'I': -0.5, 'PP': -3.5, 'A': 5.0, 'SG': 5.0, 'DD': 3.0,
        'DP': 7.0, 'GS': -2.0
    },
    2015: {
        'RB': 1.7, 'FC': -0.5, 'GC': -6.0, 'CA': -2.0, 'CV': -5.0, 'FS': 0.5, 'PE': -0.3, 'FT': 3.5,
        'FD': 1.0, 'FF': 0.7, 'G': 8.0, 'I': -0.5, 'PP': -3.5, 'A': 5.0, 'SG': 5.0, 'DD': 3.0,
        'DP': 7.0, 'GS': -2.0
    },
    2016: {
        'RB': 1.7, 'FC': -0.5, 'GC': -6.0, 'CA': -2.0, 'CV': -5.0, 'FS': 0.5, 'PE': -0.3, 'FT': 3.5,
        'FD': 1.0, 'FF': 0.7, 'G': 8.0, 'I': -0.5, 'PP': -3.5, 'A': 5.0, 'SG': 5.0, 'DD': 3.0,
        'DP': 7.0, 'GS': -2.0
    },
    2017: {
        'RB': 1.7, 'FC': -0.5, 'GC': -6.0, 'CA': -2.0, 'CV': -5.0, 'FS': 0.5, 'PE': -0.3, 'FT': 3.5,
        'FD': 1.0, 'FF': 0.7, 'G': 8.0, 'I': -0.5, 'PP': -3.5, 'A': 5.0, 'SG': 5.0, 'DD': 3.0,
        'DP': 7.0, 'GS': -2.0
    },
    # INTRODUZIU CAPITÃO
    2018: {
        'RB': 1.5, 'FC': -0.5, 'GC': -5.0, 'CA': -2.0, 'CV': -5.0, 'FS': 0.5, 'PE': -0.3, 'FT': 3.0,
        'FD': 1.2, 'FF': 0.8, 'G': 8.0, 'I': -0.5, 'PP': -4.0, 'A': 5.0, 'SG': 5.0, 'DD': 3.0,
        'DP': 7.0, 'GS': -2.0
    },
    2019: {
        'RB': 1.5, 'FC': -0.5, 'GC': -5.0, 'CA': -2.0, 'CV': -5.0, 'FS': 0.5, 'PE': -0.3, 'FT': 3.0,
        'FD': 1.2, 'FF': 0.8, 'G': 8.0, 'I': -0.5, 'PP': -4.0, 'A': 5.0, 'SG': 5.0, 'DD': 3.0,
        'DP': 7.0, 'GS': -2.0
    },
    # MUDOU O CRITÉRIO DE DEFESA DIFICIL (FICOU MENOS COMUM - Em torno de 60%)
    # ROUBADA DE BOLA VIROU DESARME (27.5% mais comum)
    2020: {
        'DS': 1.0, 'FC': -0.5, 'GC': -5.0, 'CA': -2.0, 'CV': -5.0, 'FS': 0.5, 'PI': -0.1, 'FT': 3.0,
        'FD': 1.2, 'FF': 0.8, 'G': 8.0, 'I': -0.5, 'PP': -4.0, 'A': 5.0, 'SG': 5.0, 'DD': 4.0,
        'DP': 7.0, 'GS': -2.0
    },
    # FIM DA DEFESA DIFÍCIL
    # PC E PS SÃO CRIADOS
    2021: {
        'DS': 1.0, 'FC': -0.5, 'GC': -5.0, 'CA': -2.0, 'CV': -5.0, 'FS': 0.5, 'PI': -0.1, 'FT': 3.0,
        'FD': 1.2, 'FF': 0.8, 'G': 8.0, 'I': -0.5, 'PP': -4.0, 'A': 5.0, 'SG': 5.0, 'DE': 1.0,
        'DP': 7.0, 'GS': -1.0, 'PC': -1.0, 'PS': 1.0
    },
    # FINALIZAÇÕES DE PENALTI PERDIDO ENTRAM PARA A PONTUÇÃO
    2022: {
        'DS': 1.2, 'FC': -0.3, 'GC': -3.0, 'CA': -1.0, 'CV': -3.0, 'FS': 0.5, 'PI': -0.1, 'FT': 3.0,
        'FD': 1.2, 'FF': 0.8, 'G': 8.0, 'I': -0.1, 'PP': -4.0, 'A': 5.0, 'SG': 5.0, 'DE': 1.0,
        'DP': 7.0, 'GS': -1.0, 'PC': -1.0, 'PS': 1.0
    },

    # CAPITÃO VAI DE 2X PARA 1.5X
    # PI NÃO EXISTE MAIS
    # TECNICO GANHA 1 PONTO POR VITORIA DO TIME
    2023: {
        'DS': 1.2, 'FC': -0.3, 'GC': -3.0, 'CA': -1.0, 'CV': -3.0, 'FS': 0.5, 'FT': 3.0,
        'FD': 1.2, 'FF': 0.8, 'G': 8.0, 'I': -0.1, 'PP': -4.0, 'A': 5.0, 'SG': 5.0, 'DE': 1.0,
        'DP': 7.0, 'GS': -1.0, 'PC': -1.0, 'PS': 1.0
    }
}


In [22]:
anos_rb = final_data['ano'] <= 2019

final_data.loc[anos_rb, 'DS'] = final_data.loc[anos_rb, 'RB'] * 1.275

In [23]:
final_data.groupby(['ano'])[['RB', 'DS']].sum()

,RB,DS
ano,,
2014,9636.0,12285.900
2015,9614.0,12257.850
2016,9221.0,11756.775
2017,10829.0,13806.975
2018,12012.0,15315.300
2019,9806.0,12502.650
2020,0.0,13264.000
2021,0.0,13232.000
2022,0.0,12492.000


In [24]:
def ifnull(value):
    return value if value is not None else 0

def calcular_pontuacao(row, pontuacao_por_ano):
    ano = row['ano']
    pontuacao_scouts = pontuacao_por_ano[ano]

    pontuacao = (
        row['DE'] * ifnull(pontuacao_scouts.get('DE')) +
        row['FC'] * ifnull(pontuacao_scouts.get('FC')) +
        row['GC'] * ifnull(pontuacao_scouts.get('GC')) +
        row['CA'] * ifnull(pontuacao_scouts.get('CA')) +
        row['CV'] * ifnull(pontuacao_scouts.get('CV')) +
        row['FS'] * ifnull(pontuacao_scouts.get('FS')) +
        row['PI'] * ifnull(pontuacao_scouts.get('PI')) +
        row['FT'] * ifnull(pontuacao_scouts.get('FT')) +
        row['FD'] * ifnull(pontuacao_scouts.get('FD')) +
        row['FF'] * ifnull(pontuacao_scouts.get('FF')) +
        row['G'] * ifnull(pontuacao_scouts.get('G')) +
        row['I'] * ifnull(pontuacao_scouts.get('I')) +
        row['PP'] * ifnull(pontuacao_scouts.get('PP')) +
        row['A'] * ifnull(pontuacao_scouts.get('A')) +
        row['SG'] * ifnull(pontuacao_scouts.get('SG')) +
        row['DD'] * ifnull(pontuacao_scouts.get('DD')) +
        row['DP'] * ifnull(pontuacao_scouts.get('DP')) +
        row['GS'] * ifnull(pontuacao_scouts.get('GS')) +
        row['PC'] * ifnull(pontuacao_scouts.get('PC')) +
        row['PS'] * ifnull(pontuacao_scouts.get('PS')) +
        row['RB'] * ifnull(pontuacao_scouts.get('RB')) +
        row['PE'] * ifnull(pontuacao_scouts.get('PE')) +
        row['DS'] * ifnull(pontuacao_scouts.get('DS'))
    )

    return pontuacao

In [25]:
final_data['pontos_calculado'] = final_data.fillna(0).apply(
    lambda r: calcular_pontuacao(r, pontuacao_por_ano),
    axis=1
)

In [53]:
posicoes = {
    1: 'gol',
    2: 'lat',
    3: 'zag',
    4: 'mei',
    5: 'ata',
    6: 'tec',
    'gol': 'gol',
    'lat': 'lat',
    'zag': 'zag',
    'mei': 'mei',
    'ata': 'ata',
    'tec': 'tec'
}

final_data['posicao_id'] = final_data['posicao_id'].map(posicoes)

In [54]:
final_data['dif'] = round(final_data['pontos_calculado'] - final_data['pontos_num'], 2)

erros = final_data[(final_data['dif'] != 0) & (final_data['posicao_id'] != 'tec')]

erros

,clube_id,media_num,atleta_id,status_id,clube_nome,rodada_id,preco_num,variacao_num,posicao_id,pontos_num,...,PI,DS,id,DE,PC,PS,ano,pontos_calculado,dif,clube_nome2
675,285.0,2.44,37655,NaN,internacional,27,9.83,-0.30,ata,-3.8,...,NaN,0.000,NaN,NaN,NaN,NaN,2014,3.2,7.0,internacional
2466,285.0,3.45,38219,NaN,internacional,12,12.83,0.93,mei,8.1,...,NaN,1.275,NaN,NaN,NaN,NaN,2014,16.1,8.0,internacional
3277,292.0,0.21,38579,NaN,sport,8,1.13,-0.74,mei,-2.9,...,NaN,0.000,NaN,NaN,NaN,NaN,2014,-4.9,-2.0,sport
4548,287.0,2.97,49641,NaN,vitória,12,3.91,-1.72,mei,-4.3,...,NaN,0.000,NaN,NaN,NaN,NaN,2014,-2.3,2.0,vitória
9925,NaN,0.66,68929,NaN,NaN,6,11.42,0.00,ata,0.0,...,NaN,0.000,NaN,NaN,NaN,NaN,2014,-2.0,-2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274869,277,5.07,114374,7,Santos,7,10.85,0.54,lat,6.2,...,93.0,19.000,NaN,NaN,NaN,NaN,2022,21.3,15.1,santos
275559,356,2.30,115502,6,Fortaleza,2,2.84,0.00,zag,0.0,...,2.0,1.000,NaN,NaN,NaN,NaN,2022,2.3,2.3,fortaleza
275966,314,0.55,115806,6,Avaí,2,0.97,0.31,ata,1.6,...,2.0,2.000,NaN,NaN,NaN,NaN,2022,1.1,-0.5,avaí
276468,356,0.50,116447,6,Fortaleza,2,1.65,0.00,mei,0.0,...,7.0,NaN,NaN,NaN,NaN,NaN,2022,0.5,0.5,fortaleza


In [55]:
len(final_data[(final_data['pontos_num'] != 0) | (final_data['variacao_num']) != 0])

101072

In [56]:
len(erros[(erros['pontos_num'] != 0) | (erros['variacao_num']) != 0])

9314

In [57]:
erros.groupby(['ano', 'rodada_id']).count()['atleta_id']

## RODADA 6 DE 2022 NÃO ACUMULOU COM AS ANTERIORES.

## PROBLEMAS DA DESACUMULAÇÃO ABAIXO

ano   rodada_id
2014  5             1
      6             2
      8             1
      11            1
      12            2
                   ..
2022  6            63
      7            66
      24            1
      25            1
      35            1
Name: atleta_id, Length: 95, dtype: int64

In [58]:
times = pd.read_csv('data/BRA.csv')
mapping = json.loads(open('data/mapping_clubes.json', 'r').read())

final_data['clube_nome2'] = final_data['clube_nome'].map(mapping)
missing = final_data['clube_nome'].isna()
final_data.loc[missing, 'clube_nome2'] = final_data.loc[missing, 'clube_id'].astype(str).map(
    mapping
    )

times['Home'] = times['Home'].map(mapping)
times['Away'] = times['Away'].map(mapping)

In [59]:
times[
    (times['Season'] == 2014)
].head(10)

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,PH,PD,PA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA
760,Brazil,Serie A,2014,19/04/2014,22:30,fluminense,figueirense,3.0,0.0,H,1.55,4.23,7.02,1.57,4.33,7.50,1.53,3.85,6.20
761,Brazil,Serie A,2014,19/04/2014,22:30,internacional,vitória,1.0,0.0,H,1.51,4.41,7.41,1.53,4.65,8.30,1.49,4.01,6.57
762,Brazil,Serie A,2014,20/04/2014,01:00,chapecoense,coritiba,0.0,0.0,D,2.48,3.30,3.15,2.48,3.30,3.25,2.34,3.15,3.01
763,Brazil,Serie A,2014,20/04/2014,20:00,atlético-mg,corinthians,0.0,0.0,D,2.48,3.19,3.24,2.52,3.30,3.24,2.34,3.09,3.08
764,Brazil,Serie A,2014,20/04/2014,20:00,atlético-pr,grêmio,1.0,0.0,H,3.53,3.40,2.23,3.53,3.40,2.80,3.10,3.18,2.27
765,Brazil,Serie A,2014,20/04/2014,20:00,bahia,cruzeiro,1.0,2.0,A,2.58,3.43,2.89,2.63,3.45,3.00,2.49,3.20,2.75
766,Brazil,Serie A,2014,20/04/2014,20:00,são paulo,botafogo,3.0,0.0,H,1.79,3.69,5.06,1.90,3.85,5.28,1.76,3.52,4.49
767,Brazil,Serie A,2014,20/04/2014,22:30,criciúma,palmeiras,1.0,2.0,A,3.13,3.33,2.47,3.20,3.33,2.60,2.87,3.16,2.43
768,Brazil,Serie A,2014,20/04/2014,22:30,flamengo,goiás,0.0,0.0,D,2.28,3.33,3.49,2.28,3.40,4.00,2.09,3.27,3.46
769,Brazil,Serie A,2014,20/04/2014,22:30,santos,sport,1.0,1.0,D,1.37,5.12,9.68,1.50,5.12,9.68,1.39,4.41,7.87


In [60]:
jogos = pd.read_csv('data/campeonato-brasileiro-full.csv').rename(columns={'rodata': 'rodada_id'})

In [61]:
jogos

,ID,rodada_id,data,hora,mandante,visitante,formacao_mandante,formacao_visitante,tecnico_mandante,tecnico_visitante,vencedor,arena,mandante_Placar,visitante_Placar,mandante_Estado,visitante_Estado
0,1,1,29/3/2003,16:00,Guarani,Vasco,NaN,NaN,NaN,NaN,Guarani,Brinco de Ouro,4,2,SP,RJ
1,2,1,29/3/2003,16:00,Athletico-PR,Gremio,NaN,NaN,NaN,NaN,Athletico-PR,Arena da Baixada,2,0,PR,RS
2,3,1,30/3/2003,16:00,Flamengo,Coritiba,NaN,NaN,NaN,NaN,-,Maracanã,1,1,RJ,PR
3,4,1,30/3/2003,16:00,Goias,Paysandu,NaN,NaN,NaN,NaN,-,Serra Dourada,2,2,GO,PA
4,5,1,30/3/2003,16:00,Internacional,Ponte Preta,NaN,NaN,NaN,NaN,-,Beira Rio,1,1,RS,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8020,8021,38,13/11/2022,16:03,Cuiaba,Coritiba,4-1-4-1,4-1-4-1,A. Cardoso de Oliveira,A. Ferreira,Cuiaba,Arena Pantanal,2,1,MT,PR
8021,8022,38,13/11/2022,16:03,Bragantino,Fluminense,4-2-3-1,4-2-3-1,M. Nogueira Barbieri,F. Diniz Silva,Fluminense,Nabizão,0,1,SP,RJ
8022,8023,38,13/11/2022,16:03,Corinthians,Atletico-MG,4-1-4-1,4-2-3-1,F. J. Monteiro Almeida,A. Stival,Atletico-MG,Neo Química Arena,0,1,SP,MG
8023,8024,38,13/11/2022,16:03,Internacional,Palmeiras,4-2-3-1,4-2-3-1,L. Venker de Menezes,A. Moreira Ferreira,Internacional,Estádio José Pinheiro Borda,3,0,RS,SP


In [62]:
jogos['mandante'] = jogos['mandante'].map(mapping)
jogos['visitante'] = jogos['visitante'].map(mapping)

In [63]:
jogos.loc[4606::][jogos.loc[4606::]['mandante'].isna()]

,ID,rodada_id,data,hora,mandante,visitante,formacao_mandante,formacao_visitante,tecnico_mandante,tecnico_visitante,vencedor,arena,mandante_Placar,visitante_Placar,mandante_Estado,visitante_Estado


In [64]:
rodadas = 10*[i for i in range(1, 39)]
ano = 2014

for index, row in jogos.loc[4606::].iterrows():
    jogos.loc[index, 'ano'] = ano
    rodadas.remove(row['rodada_id'])
    if len(rodadas) == 0 or len(rodadas) == 1 and ano == 2016:
        ano += 1
        rodadas = 10*[i for i in range(1, 39)]


In [65]:
jogos.loc[4606::][['mandante', 'visitante']].describe()

,mandante,visitante
count,3419,3419
unique,34,34
top,fluminense,palmeiras
freq,171,171


In [66]:
times = times.merge(
    jogos[['ano', 'mandante', 'visitante', 'rodada_id']],
    left_on=['Home', 'Away', 'Season'],
    right_on=['mandante', 'visitante', 'ano'],
    how='left'
)

In [67]:
times

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,...,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,ano,mandante,visitante,rodada_id
0,Brazil,Serie A,2012,19/05/2012,22:30,palmeiras,portuguesa,1.0,1.0,D,...,1.76,3.87,5.31,1.69,3.50,4.90,NaN,NaN,NaN,NaN
1,Brazil,Serie A,2012,19/05/2012,22:30,sport,flamengo,1.0,1.0,D,...,2.83,3.42,2.70,2.59,3.23,2.58,NaN,NaN,NaN,NaN
2,Brazil,Serie A,2012,20/05/2012,01:00,figueirense,náutico,2.0,1.0,H,...,1.67,4.05,7.22,1.59,3.67,5.64,NaN,NaN,NaN,NaN
3,Brazil,Serie A,2012,20/05/2012,20:00,botafogo,são paulo,4.0,2.0,H,...,2.49,3.39,3.15,2.35,3.26,2.84,NaN,NaN,NaN,NaN
4,Brazil,Serie A,2012,20/05/2012,20:00,corinthians,fluminense,0.0,1.0,A,...,1.96,3.53,4.41,1.89,3.33,3.89,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4175,Brazil,Serie A,2022,13/11/2022,19:00,corinthians,atlético-mg,0.0,1.0,A,...,3.35,3.34,2.57,3.06,3.07,2.45,2022.0,corinthians,atlético-mg,38.0
4176,Brazil,Serie A,2022,13/11/2022,19:00,cuiabá,coritiba,2.0,1.0,H,...,2.05,3.60,4.35,1.94,3.37,4.00,2022.0,cuiabá,coritiba,38.0
4177,Brazil,Serie A,2022,13/11/2022,19:00,goiás,são paulo,0.0,4.0,A,...,5.25,3.99,1.75,4.85,3.78,1.69,2022.0,goiás,são paulo,38.0
4178,Brazil,Serie A,2022,13/11/2022,19:00,internacional,palmeiras,3.0,0.0,H,...,2.64,3.66,2.85,2.49,3.45,2.72,2022.0,internacional,palmeiras,38.0


In [68]:
times = times[[
    'Date', 'Time', 'mandante', 'visitante', 'HG', 'AG', 'AvgH', 'AvgD', 'AvgA', 'ano', 'rodada_id'
]].dropna().astype({
    'HG': int,
    'AG': int,
    'ano': int,
    'rodada_id': int
})

In [69]:
def adicionar_odds(df_jogadores, df_odds):
    # Primeiro, faça o merge dos DataFrames usando 'ano' e 'rodada_id' como chave

    dfs = []  # list to temporarily store partially merged dfs
    df_jogadores.reset_index(inplace=True, drop=True)  # reset index to maintain correct index order
    for col in ['mandante', 'visitante']:  # iterate over columns
        # append partially merged columns
        dfs.append(pd.merge(df_jogadores, df_odds, left_on=['ano', 'rodada_id', 'clube_nome2'], right_on=['ano', 'rodada_id', col]))
    # concat all partial results:
    df_merged = pd.concat(dfs, axis=0)  # set old index

    # Calcule odds_vitoria, odds_derrota, gols_time e gols_oponente usando operações vetorizadas
    df_merged['casa'] = df_merged.apply(lambda row: 1 if row['clube_nome2'] == row['mandante'] else 0, axis=1)
    df_merged['adversario_nome'] = df_merged.apply(lambda row: row['visitante'] if row['clube_nome2'] == row['mandante'] else row['mandante'], axis=1)
    df_merged['odds_vitoria'] = df_merged.apply(lambda row: row['AvgH'] if row['clube_nome2'] == row['mandante'] else row['AvgA'], axis=1)
    df_merged['odds_derrota'] = df_merged.apply(lambda row: row['AvgA'] if row['clube_nome2'] == row['mandante'] else row['AvgH'], axis=1)
    df_merged['gols_time'] = df_merged.apply(lambda row: row['HG'] if row['clube_nome2'] == row['mandante'] else row['AG'], axis=1)
    df_merged['gols_oponente'] = df_merged.apply(lambda row: row['AG'] if row['clube_nome2'] == row['mandante'] else row['HG'], axis=1)

    # Adicione a coluna odds_empate
    df_merged['odds_empate'] = df_merged['AvgD']

    # Remova as colunas 'mandante', 'visitante', 'AvgH', 'AvgA', 'AvgD', 'HG' e 'AG', pois não são mais necessárias
    df_merged.drop(['mandante', 'visitante', 'AvgH', 'AvgA', 'AvgD', 'HG', 'AG'], axis=1, inplace=True)

    df_merged['clube_nome'] = df_merged['clube_nome2']
    df_merged.drop(['clube_nome2'], axis=1, inplace=True)

    return df_merged

In [70]:
final_data_odds = adicionar_odds(df_jogadores=final_data, df_odds=times)

In [71]:
final_data_odds[
    (((final_data_odds['pontos_num'] != 0) | (final_data_odds['variacao_num']) != 0) &
    ((final_data_odds['dif'] == 0))) |
    (final_data_odds['posicao_id'] == 'tec')
    ]

,clube_id,media_num,atleta_id,status_id,clube_nome,rodada_id,preco_num,variacao_num,posicao_id,pontos_num,...,dif,Date,Time,casa,adversario_nome,odds_vitoria,odds_derrota,gols_time,gols_oponente,odds_empate
0,285.0,5.00,36443,NaN,internacional,1,10.60,1.60,gol,5.00,...,0.00,19/04/2014,22:30,1,vitória,1.49,6.57,1,0,4.01
1,285.0,2.00,36540,NaN,internacional,1,12.88,-2.12,zag,2.00,...,0.00,19/04/2014,22:30,1,vitória,1.49,6.57,1,0,4.01
2,285.0,0.50,37655,NaN,internacional,1,6.44,-1.56,ata,0.50,...,0.00,19/04/2014,22:30,1,vitória,1.49,6.57,1,0,4.01
3,285.0,1.70,38074,NaN,internacional,1,3.56,0.56,mei,1.70,...,0.00,19/04/2014,22:30,1,vitória,1.49,6.57,1,0,4.01
4,285.0,0.20,38108,NaN,internacional,1,10.00,-3.00,ata,0.20,...,-0.00,19/04/2014,22:30,1,vitória,1.49,6.57,1,0,4.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130023,277,2.91,110415,7,santos,37,5.28,-0.15,ata,0.80,...,-0.00,10/11/2022,23:00,0,botafogo,3.37,2.21,0,3,3.24
130026,277,2.84,112886,7,santos,37,3.07,-0.41,mei,0.50,...,0.00,10/11/2022,23:00,0,botafogo,3.37,2.21,0,3,3.24
130028,277,2.66,114374,7,santos,37,7.74,0.00,lat,1.10,...,-0.00,10/11/2022,23:00,0,botafogo,3.37,2.21,0,3,3.24
130030,277,2.40,115404,6,santos,37,3.05,-0.20,ata,1.10,...,0.00,10/11/2022,23:00,0,botafogo,3.37,2.21,0,3,3.24


In [72]:
def calc_soma_fd_por_time_e_rodada(df):
    return df.groupby(['ano', 'rodada_id', 'clube_nome'])['FD'].sum().reset_index()

def zerar_de_nao_goleiros(row):
    if row['posicao_id'] != 'gol':
        row['DE'] = 0
    return row

def adicionar_fd_adversario(df_original, soma_fd_por_time_e_rodada):
    # Renomeando as colunas do DataFrame com a soma das FD para fazer o merge
    soma_fd_por_time_e_rodada = soma_fd_por_time_e_rodada.rename(columns={'clube_nome': 'adversario_nome', 'FD': 'DE_calc'})

    # Fazendo o merge com o DataFrame original
    df_final = pd.merge(df_original, soma_fd_por_time_e_rodada, on=['ano', 'rodada_id', 'adversario_nome'])

    # Atualizando a coluna 'DE' para 0 quando 'posicao_id' != 'gol'
    df_final['DE'] = df_final['DE_calc']
    df_final.drop(['DE_calc'], axis=1, inplace=True)

    df_final = df_final.apply(zerar_de_nao_goleiros, axis=1)

    return df_final

In [73]:
soma_fd_por_time_e_rodada = calc_soma_fd_por_time_e_rodada(final_data_odds)

final_data_odds = adicionar_fd_adversario(final_data_odds, soma_fd_por_time_e_rodada)

In [74]:
final_data_odds[['ano', 'rodada_id', 'posicao_id', 'DE']]

,ano,rodada_id,posicao_id,DE
0,2014,1,gol,2.0
1,2014,1,zag,0.0
2,2014,1,ata,0.0
3,2014,1,mei,0.0
4,2014,1,ata,0.0
...,...,...,...,...
259985,2022,37,ata,0.0
259986,2022,37,mei,0.0
259987,2022,37,mei,0.0
259988,2022,37,zag,0.0


In [75]:
dados_uteis = final_data_odds[
    (((final_data_odds['pontos_num'] != 0) | (final_data_odds['variacao_num']) != 0) &
    ((final_data_odds['dif'] == 0))) |
    (final_data_odds['posicao_id'] == 'tec')
    ]

In [76]:
dados_uteis['posicao_id'].value_counts()

mei    31132
ata    21714
lat    13484
zag    13144
tec     6838
gol     5865
Name: posicao_id, dtype: int64

In [77]:
dados_uteis = dados_uteis[[
    'preco_num', 'variacao_num', 'atleta_id', 'status_id',
    'clube_nome', 'pontos_num', 'rodada_id', 'apelido',
    'posicao_id', 'FS', 'A', 'FT', 'FD', 'FF', 'G', 'I', 'PP',
    'FC', 'GC', 'CA', 'CV', 'SG', 'DD', 'DP', 'GS', 'DS', 'DE',
    'PC', 'PS', 'ano', 'Date', 'Time', 'casa',
    'adversario_nome', 'odds_vitoria', 'odds_derrota', 'gols_time',
    'gols_oponente', 'odds_empate'
]]

for ano in anos:
    dados_uteis[dados_uteis['ano'] == ano].to_csv(f'raw_data_{ano}.csv', index=False)
